In [1]:
from IPython.display import clear_output
!wget https://disk.karelia.pro/djnSMfz/Трек%201.%20Онежское%20озеро.tgz?attachment
import os
try:
    os.mkdir("./data")
except: pass
!tar -xvzf "Трек 1. Онежское озеро.tgz?attachment" -C ./data
clear_output()

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!pip install catboost

Mounted at /content/drive
     |████████████████████████████████| 65.7MB 46kB/s 


In [3]:
path = "./drive/My Drive/Ситилинк Хакатон/"

In [4]:
import os
from torchvision import transforms
import pickle
import pandas as pd
import torch
from tqdm.notebook import tqdm
from PIL import Image
import numpy as np
 
class LakeClassifier():
    def __init__(self, models_path):
        self.device = torch.device('cuda' if 
                                   torch.cuda.is_available() else 'cpu')
        self.network = torch.load(os.path.join(models_path, "network.pth")).to(self.device)
        self.transform = transforms.ToTensor()
        self.size = (128, 128)
    
    def predict_network(self, path):
        predictions = []
        for filename in tqdm(os.listdir(path)):
            with Image.open(os.path.join(path, filename)) as image:
                image = image.resize(self.size)
            X = self.transform(image)
            X = X.view(1, *X.size()).to(self.device)
            prediction = self.network(X).argmax(dim=1).item()
            predictions.append([filename, prediction])
        return predictions

In [5]:
model_nn = LakeClassifier(path)

In [6]:
predictions = model_nn.predict_network("data")

In [7]:
import pandas as pd

# переводим дату в числа
def date_to_data(frame, holidays=None,
                 drop=False, name="date", suffix=""):
    assert name in frame, "Нужна колонка с датой"
    date = frame[name].dt

    info = pd.concat([date.days_in_month,
                     date.month,
                     date.quarter,
                     date.month % 12 // 3 + 1], axis=1)
    
    info.columns = ["days_in_month", "month",
                    "quarter", "season"]

    temp = pd.concat([date.isocalendar(),
                      info], axis=1)

    temp.columns = temp.columns + suffix
    frame = pd.concat([frame, temp], axis=1)
    if drop:
        return frame.drop([name], axis=1)
    return frame

df = pd.read_csv(path + "joined_last.csv", index_col="Unnamed: 0")
df["date"] = pd.to_datetime(df["name"].apply(lambda x: x[:-4]))
df = date_to_data(df, name="date", drop=False)

In [8]:
predictions_df = pd.DataFrame(predictions, columns=["filename", "prediction"])

In [9]:
frame = df.merge(predictions_df, left_on="name", right_on="filename")

In [10]:
names = ['hour', 'year', 'week', 'day',
        'days_in_month', 'month', 'quarter', 'season',
        'prediction']

In [11]:
from sklearn.model_selection import train_test_split

def train_test_val_split(X, y,
                         test_size=0.3, val_size=0.3,
                         random_state=42, shuffle=True):
    """
    separator for train, test, validation
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        train_size=1 - (test_size + val_size),
                                                        random_state=random_state, shuffle=shuffle)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test,
                                                    test_size=test_size / (test_size + val_size),
                                                    random_state=random_state, shuffle=shuffle)
    return X_train, y_train, X_test, y_test, X_val, y_val

X = frame[names].astype(np.float32)
y = frame["y"].astype(np.int32)

X_train, y_train, X_test, y_test, X_val, y_val = train_test_val_split(X, y,
                                                                      test_size=0.3,
                                                                      val_size=0.3,
                                                                      shuffle=False)

In [12]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(random_state=42,
                         verbose=1,
                         iterations=1000,
                         grow_policy="Depthwise",
                         loss_function="CrossEntropy",
                         task_type="GPU")

# clf.fit(X_train, y_train)
clf.fit(X, y)

0:	learn: 0.5823062	total: 13.1ms	remaining: 13.1s
1:	learn: 0.4933935	total: 27.9ms	remaining: 13.9s
2:	learn: 0.4309658	total: 41.7ms	remaining: 13.9s
3:	learn: 0.3681565	total: 53.4ms	remaining: 13.3s
4:	learn: 0.3144649	total: 65.3ms	remaining: 13s
5:	learn: 0.2709536	total: 77.2ms	remaining: 12.8s
6:	learn: 0.2348016	total: 89.2ms	remaining: 12.6s
7:	learn: 0.2135112	total: 101ms	remaining: 12.5s
8:	learn: 0.1857947	total: 113ms	remaining: 12.4s
9:	learn: 0.1630686	total: 125ms	remaining: 12.3s
10:	learn: 0.1433447	total: 136ms	remaining: 12.3s
11:	learn: 0.1274551	total: 148ms	remaining: 12.2s
12:	learn: 0.1176302	total: 164ms	remaining: 12.5s
13:	learn: 0.1113544	total: 176ms	remaining: 12.4s
14:	learn: 0.1001070	total: 188ms	remaining: 12.3s
15:	learn: 0.0914185	total: 201ms	remaining: 12.4s
16:	learn: 0.0852495	total: 217ms	remaining: 12.6s
17:	learn: 0.0805342	total: 229ms	remaining: 12.5s
18:	learn: 0.0738700	total: 241ms	remaining: 12.4s
19:	learn: 0.0678654	total: 253ms	re

In [13]:
# from sklearn.metrics import accuracy_score
# accuracy_score(y_val, clf.predict(X_val))

In [14]:
# from sklearn.metrics import accuracy_score
# accuracy_score(y_test, clf.predict(X_test))

In [15]:
sorted(zip(clf.feature_names_, clf.feature_importances_),
       reverse=True, key=lambda x: x[1])

[('week', 67.83276471613571),
 ('prediction', 13.779804894990063),
 ('year', 10.302681003145342),
 ('month', 3.9205503150391583),
 ('days_in_month', 1.81936273272006),
 ('day', 1.2875943101704015),
 ('hour', 0.5294438344404219),
 ('season', 0.5054627492915942),
 ('quarter', 0.022335444067244203)]

In [16]:
import pickle
with open(path + "ModernClassifier.pkl", "wb") as f:
    pickle.dump(clf, f)